# Random Forest for Customer Churn Prediction

This notebook evaluates Random Forest as an ensemble-based classification model
for customer churn prediction and compares its performance to previous models.

**1. Imports**

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

**2. Load data**

In [2]:
df = pd.read_csv("../data/raw_data.csv")
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

X = df.drop("Churn", axis=1)
y = df["Churn"].map({"Yes": 1, "No": 0})

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

## Data Preparation

The same dataset split used in previous experiments is applied here to ensure
a fair comparison across models.    

**3. Preprocessing pipeline**

In [3]:
num_features = ["tenure", "MonthlyCharges", "TotalCharges"]
cat_features = [col for col in X.columns if col not in num_features]

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ]
)

The preprocessing pipeline is reused without modification to maintain consistency
with previous models.

**4. Random Forest model**

## Random Forest Model

Random Forest is evaluated as a robust ensemble method that aggregates multiple
decision trees to reduce variance and improve generalization.

In [4]:
rf_model = Pipeline(
    steps=[
        ("preprocessing", preprocessor),
        ("model", RandomForestClassifier(
            n_estimators=300,
            max_depth=None,
            min_samples_split=2,
            min_samples_leaf=1,
            random_state=42,
            n_jobs=-1
        ))
    ]
)

rf_model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers c

**5. Evaluation**

In [6]:
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

roc_auc_rf = roc_auc_score(y_test, y_prob_rf)
roc_auc_rf

0.8290010075176315

In [7]:
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.83      0.91      0.87      1035
           1       0.66      0.49      0.56       374

    accuracy                           0.80      1409
   macro avg       0.74      0.70      0.71      1409
weighted avg       0.78      0.80      0.79      1409



### Evaluation Results

The Random Forest model achieves a ROC-AUC of 0.829, which is lower than both
the logistic regression baseline and the Gradient Boosting model. While overall
accuracy remains comparable (0.80), the model demonstrates reduced sensitivity
to churned customers.

In particular, recall for the churn class is 0.49, indicating that nearly half
of churned customers are not detected at the default classification threshold.
This suggests that the Random Forest model is relatively conservative, favoring
the majority (non-churn) class and prioritizing stability over sensitivity to
minority class detection.

Compared to Gradient Boosting, Random Forest provides weaker ranking performance
and lower churn recall in this setting. As a result, Gradient Boosting remains
the strongest model among the evaluated ensemble methods for this dataset.

**6. Interpretation**

### Interpretation

The Random Forest model demonstrates strong performance in identifying
non-churned customers, as indicated by a high recall for the majority class.
However, its ability to detect churned customers is more limited.

This behavior suggests that the model is relatively conservative: it prioritizes
correct classification of loyal customers while being less sensitive to churn
signals. Such a pattern is common for Random Forest models trained on imbalanced
datasets, where the majority class dominates the learning process.

While Random Forest is capable of capturing non-linear relationships and complex
feature interactions, in this configuration it does not effectively emphasize
the minority churn class without additional tuning.

**7. Comparison with Previous Models**

### Comparison with Previous Models

Compared to the logistic regression baseline, Random Forest does not provide an
improvement in either ROC-AUC or churn recall. While overall accuracy remains
similar, the model exhibits lower sensitivity to churned customers.

Relative to Gradient Boosting, Random Forest shows weaker discriminative ability
(ROC-AUC) and the lowest recall for the churn class among the evaluated models.
This indicates that Gradient Boosting captures churn-related patterns more
effectively in this dataset.

These results highlight that increased model complexity does not automatically
translate into better performance for minority class detection.

**8. Summary**

### Summary

In this experiment, Random Forest was evaluated as an alternative ensemble method
for customer churn prediction. Although the model achieves stable overall
performance, it underperforms both logistic regression and Gradient Boosting in
terms of ROC-AUC and churn recall.

The findings suggest that Random Forest, in its current configuration, is less
suited for identifying churned customers in this dataset. Consequently,
Gradient Boosting remains the most effective model among those evaluated and
serves as the primary candidate for further optimization and threshold tuning.